# Lab4
## Zad 1

In [0]:
from pyspark.sql.types import *
from pyspark.sql.functions import to_timestamp, coalesce, col

# Wczytanie danych CSV
data = spark.read.format("csv")\
    .option("inferSchema", "true")\
    .option("header", "true")\
    .load("/FileStore/tables/201508_trip_data.csv")

# Uporządkowane formaty daty
pat1 = "M/d/yyyy H:mm"
pat2 = "M/d/yyyy HH:mm"
pat3 = "M/dd/yyyy H:mm"
pat4 = "M/dd/yyyy HH:mm"
pat5 = "MM/d/yyyy H:mm"
pat6 = "MM/d/yyyy HH:mm"
pat7 = "MM/dd/yyyy H:mm"
pat8 = "MM/dd/yyyy HH:mm"

# Parsowanie dat + konwersja Zip Code
df = data.withColumn("Start Date", coalesce(
        to_timestamp("Start Date", pat1),
        to_timestamp("Start Date", pat2),
        to_timestamp("Start Date", pat3),
        to_timestamp("Start Date", pat4),
        to_timestamp("Start Date", pat5),
        to_timestamp("Start Date", pat6),
        to_timestamp("Start Date", pat7),
        to_timestamp("Start Date", pat8)
    )
).withColumn("End Date", coalesce(
        to_timestamp("End Date", pat1),
        to_timestamp("End Date", pat2),
        to_timestamp("End Date", pat3),
        to_timestamp("End Date", pat4),
        to_timestamp("End Date", pat5),
        to_timestamp("End Date", pat6),
        to_timestamp("End Date", pat7),
        to_timestamp("End Date", pat8)
    )
).withColumn("Zip Code", col("Zip Code").cast(IntegerType()))

# Wyświetlenie
df.show(50) 


+-------+--------+-------------------+--------------------+--------------+-------------------+--------------------+------------+------+---------------+--------+
|Trip ID|Duration|         Start Date|       Start Station|Start Terminal|           End Date|         End Station|End Terminal|Bike #|Subscriber Type|Zip Code|
+-------+--------+-------------------+--------------------+--------------+-------------------+--------------------+------------+------+---------------+--------+
| 913460|     765|2015-08-31 23:26:00|Harry Bridges Pla...|            50|2015-08-31 23:39:00|San Francisco Cal...|          70|   288|     Subscriber|    2139|
| 913459|    1036|2015-08-31 23:11:00|San Antonio Shopp...|            31|2015-08-31 23:28:00|Mountain View Cit...|          27|    35|     Subscriber|   95032|
| 913455|     307|2015-08-31 23:13:00|      Post at Kearny|            47|2015-08-31 23:18:00|   2nd at South Park|          64|   468|     Subscriber|   94107|
| 913454|     409|2015-08-31 23:10

In [0]:
#wiersze z nullami (przykład)

nullDf = df.filter(col("Zip Code").isNull())

nullDf.show(50)


+-------+--------+-------------------+--------------------+--------------+-------------------+--------------------+------------+------+---------------+--------+
|Trip ID|Duration|         Start Date|       Start Station|Start Terminal|           End Date|         End Station|End Terminal|Bike #|Subscriber Type|Zip Code|
+-------+--------+-------------------+--------------------+--------------+-------------------+--------------------+------------+------+---------------+--------+
| 913084|     547|2015-08-31 17:58:00|       Howard at 2nd|            63|2015-08-31 18:08:00|Embarcadero at Sa...|          60|   279|       Customer|    null|
| 912988|     293|2015-08-31 17:36:00|   Steuart at Market|            74|2015-08-31 17:41:00|       Howard at 2nd|          63|   282|       Customer|    null|
| 912665|     824|2015-08-31 16:14:00|Embarcadero at Sa...|            60|2015-08-31 16:28:00|Harry Bridges Pla...|          50|   511|       Customer|    null|
| 912664|     827|2015-08-31 16:14

### Użycie wybranych funkcji

In [0]:
from pyspark.sql.functions import (explode, regexp_replace, regexp_extract, array_contains, lit)

In [0]:
#fill

from random import randint

filledDf = nullDf.na.fill(randint(11,100))

filledDf.show(50)

+-------+--------+-------------------+--------------------+--------------+-------------------+--------------------+------------+------+---------------+--------+
|Trip ID|Duration|         Start Date|       Start Station|Start Terminal|           End Date|         End Station|End Terminal|Bike #|Subscriber Type|Zip Code|
+-------+--------+-------------------+--------------------+--------------+-------------------+--------------------+------------+------+---------------+--------+
| 913084|     547|2015-08-31 17:58:00|       Howard at 2nd|            63|2015-08-31 18:08:00|Embarcadero at Sa...|          60|   279|       Customer|      88|
| 912988|     293|2015-08-31 17:36:00|   Steuart at Market|            74|2015-08-31 17:41:00|       Howard at 2nd|          63|   282|       Customer|      88|
| 912665|     824|2015-08-31 16:14:00|Embarcadero at Sa...|            60|2015-08-31 16:28:00|Harry Bridges Pla...|          50|   511|       Customer|      88|
| 912664|     827|2015-08-31 16:14

In [0]:
#explode
from pyspark.sql.functions import split
arrDf = df.select("trip ID", split(col("Start Station"), " ").alias("Start Station"))
expDf = arrDf.select("trip ID", explode("Start Station"))
expDf.show(50)

+-------+-----------+
|trip ID|        col|
+-------+-----------+
| 913460|      Harry|
| 913460|    Bridges|
| 913460|      Plaza|
| 913460|     (Ferry|
| 913460|  Building)|
| 913459|        San|
| 913459|    Antonio|
| 913459|   Shopping|
| 913459|     Center|
| 913455|       Post|
| 913455|         at|
| 913455|     Kearny|
| 913454|        San|
| 913454|       Jose|
| 913454|       City|
| 913454|       Hall|
| 913453|Embarcadero|
| 913453|         at|
| 913453|     Folsom|
| 913452|      Yerba|
| 913452|      Buena|
| 913452|     Center|
| 913452|         of|
| 913452|        the|
| 913452|       Arts|
| 913452|       (3rd|
| 913452|          @|
| 913452|    Howard)|
| 913451|Embarcadero|
| 913451|         at|
| 913451|     Folsom|
| 913450|Embarcadero|
| 913450|         at|
| 913450|    Sansome|
| 913449|      Beale|
| 913449|         at|
| 913449|     Market|
| 913448|       Post|
| 913448|         at|
| 913448|     Kearny|
| 913443|Embarcadero|
| 913443|         at|
| 913443| 

In [0]:
#drop
droppedDf = df.na.drop()
droppedDf.show(50)

+-------+--------+-------------------+--------------------+--------------+-------------------+--------------------+------------+------+---------------+--------+
|Trip ID|Duration|         Start Date|       Start Station|Start Terminal|           End Date|         End Station|End Terminal|Bike #|Subscriber Type|Zip Code|
+-------+--------+-------------------+--------------------+--------------+-------------------+--------------------+------------+------+---------------+--------+
| 913460|     765|2015-08-31 23:26:00|Harry Bridges Pla...|            50|2015-08-31 23:39:00|San Francisco Cal...|          70|   288|     Subscriber|    2139|
| 913459|    1036|2015-08-31 23:11:00|San Antonio Shopp...|            31|2015-08-31 23:28:00|Mountain View Cit...|          27|    35|     Subscriber|   95032|
| 913455|     307|2015-08-31 23:13:00|      Post at Kearny|            47|2015-08-31 23:18:00|   2nd at South Park|          64|   468|     Subscriber|   94107|
| 913454|     409|2015-08-31 23:10

In [0]:
from pyspark.sql.functions import regexp_replace

regex = r"\d{4}"  # znajdź rok (cztery cyfry)

rDf = df.select(
    "Start Date",
    regexp_replace("Start Date", regex, "1900").alias("regexp_replace")
)

rDf.show(50)


+-------------------+-------------------+
|         Start Date|     regexp_replace|
+-------------------+-------------------+
|2015-08-31 23:26:00|1900-08-31 23:26:00|
|2015-08-31 23:11:00|1900-08-31 23:11:00|
|2015-08-31 23:13:00|1900-08-31 23:13:00|
|2015-08-31 23:10:00|1900-08-31 23:10:00|
|2015-08-31 23:09:00|1900-08-31 23:09:00|
|2015-08-31 23:07:00|1900-08-31 23:07:00|
|2015-08-31 23:07:00|1900-08-31 23:07:00|
|2015-08-31 22:16:00|1900-08-31 22:16:00|
|2015-08-31 22:12:00|1900-08-31 22:12:00|
|2015-08-31 21:57:00|1900-08-31 21:57:00|
|2015-08-31 21:49:00|1900-08-31 21:49:00|
|2015-08-31 21:44:00|1900-08-31 21:44:00|
|2015-08-31 21:39:00|1900-08-31 21:39:00|
|2015-08-31 21:31:00|1900-08-31 21:31:00|
|2015-08-31 21:25:00|1900-08-31 21:25:00|
|2015-08-31 21:19:00|1900-08-31 21:19:00|
|2015-08-31 21:17:00|1900-08-31 21:17:00|
|2015-08-31 21:16:00|1900-08-31 21:16:00|
|2015-08-31 21:11:00|1900-08-31 21:11:00|
|2015-08-31 21:07:00|1900-08-31 21:07:00|
|2015-08-31 21:06:00|1900-08-31 21

In [0]:
#regexp_extract

regex2 = r"(\d{4})-(\d{1,2})"

reDf = rDf.select(regexp_extract("regexp_replace", regex2, 2).alias("extract"))

reDf.show(50)

+-------+
|extract|
+-------+
|     08|
|     08|
|     08|
|     08|
|     08|
|     08|
|     08|
|     08|
|     08|
|     08|
|     08|
|     08|
|     08|
|     08|
|     08|
|     08|
|     08|
|     08|
|     08|
|     08|
|     08|
|     08|
|     08|
|     08|
|     08|
|     08|
|     08|
|     08|
|     08|
|     08|
|     08|
|     08|
|     08|
|     08|
|     08|
|     08|
|     08|
|     08|
|     08|
|     08|
|     08|
|     08|
|     08|
|     08|
|     08|
|     08|
|     08|
|     08|
|     08|
|     08|
+-------+
only showing top 50 rows



In [0]:
from pyspark.sql.functions import coalesce

ifnDf = df.select(
    "Zip Code",
    coalesce(col("Zip Code"), col("Duration")).alias("test")  # jeśli Zip Code jest null, użyj Duration
).filter(col("Zip Code").isNull())

ifnDf.show(50)

+--------+-----+
|Zip Code| test|
+--------+-----+
|    null|  547|
|    null|  293|
|    null|  824|
|    null|  827|
|    null|  324|
|    null| 2092|
|    null|  519|
|    null|  744|
|    null|  818|
|    null| 1502|
|    null|  492|
|    null|  545|
|    null| 3818|
|    null|20958|
|    null| 4424|
|    null|21180|
|    null|21182|
|    null| 4652|
|    null| 5254|
|    null| 1447|
|    null| 1495|
|    null|  676|
|    null|  801|
|    null|  940|
|    null| 4586|
|    null|18353|
|    null| 9581|
|    null| 9791|
|    null| 6481|
|    null| 1710|
|    null| 1728|
|    null| 1812|
|    null|  538|
|    null|  725|
|    null| 1840|
|    null| 1925|
|    null| 7114|
|    null|  668|
|    null| 2015|
|    null|  685|
|    null|  863|
|    null|  873|
|    null|  342|
|    null|  632|
|    null|  648|
|    null| 5444|
|    null|  432|
|    null|  444|
|    null|  959|
|    null|  494|
+--------+-----+
only showing top 50 rows



In [0]:
from pyspark.sql.functions import when, col

# Do testu – Duration2 czasem różni się od Duration
sampDf = df.select(
    "Duration",
    (when(col("Duration") % 2 == 0, 2 * col("Duration"))
     .otherwise(col("Duration"))).alias("Duration2")
)

# NULLIF(Duration, Duration2)
niDf = sampDf.select(
    "Duration",
    "Duration2",
    (when(col("Duration") == col("Duration2"), None)
     .otherwise(col("Duration"))).alias("nullif")
)

niDf.show(50)


+--------+---------+------+
|Duration|Duration2|nullif|
+--------+---------+------+
|     765|      765|  null|
|    1036|     2072|  1036|
|     307|      307|  null|
|     409|      409|  null|
|     789|      789|  null|
|     293|      293|  null|
|     896|     1792|   896|
|     255|      255|  null|
|     126|      252|   126|
|     932|     1864|   932|
|     691|      691|  null|
|     633|      633|  null|
|     387|      387|  null|
|     281|      281|  null|
|     424|      848|   424|
|     283|      283|  null|
|     145|      145|  null|
|     703|      703|  null|
|     605|      605|  null|
|     902|     1804|   902|
|     481|      481|  null|
|     335|      335|  null|
|     704|     1408|   704|
|     326|      652|   326|
|     274|      548|   274|
|     709|      709|  null|
|     486|      972|   486|
|     273|      273|  null|
|     497|      497|  null|
|     395|      395|  null|
|     509|      509|  null|
|     671|      671|  null|
|     307|      307|

In [0]:
#replace

repDf = niDf.na.replace(25, 33, ['Duration'])

repDf.show(50)

+--------+---------+------+
|Duration|Duration2|nullif|
+--------+---------+------+
|     765|      765|  null|
|    1036|     2072|  1036|
|     307|      307|  null|
|     409|      409|  null|
|     789|      789|  null|
|     293|      293|  null|
|     896|     1792|   896|
|     255|      255|  null|
|     126|      252|   126|
|     932|     1864|   932|
|     691|      691|  null|
|     633|      633|  null|
|     387|      387|  null|
|     281|      281|  null|
|     424|      848|   424|
|     283|      283|  null|
|     145|      145|  null|
|     703|      703|  null|
|     605|      605|  null|
|     902|     1804|   902|
|     481|      481|  null|
|     335|      335|  null|
|     704|     1408|   704|
|     326|      652|   326|
|     274|      548|   274|
|     709|      709|  null|
|     486|      972|   486|
|     273|      273|  null|
|     497|      497|  null|
|     395|      395|  null|
|     509|      509|  null|
|     671|      671|  null|
|     307|      307|

In [0]:
#array_contains
aDf = arrDf.select("Start Station", array_contains("Start Station", "San").alias('arr_cont'))

aDf.show(50)

+--------------------+--------+
|       Start Station|arr_cont|
+--------------------+--------+
|[Harry, Bridges, ...|   false|
|[San, Antonio, Sh...|    true|
|  [Post, at, Kearny]|   false|
|[San, Jose, City,...|    true|
|[Embarcadero, at,...|   false|
|[Yerba, Buena, Ce...|   false|
|[Embarcadero, at,...|   false|
|[Embarcadero, at,...|   false|
| [Beale, at, Market]|   false|
|  [Post, at, Kearny]|   false|
|[Embarcadero, at,...|   false|
|  [Market, at, 10th]|   false|
|   [Market, at, 4th]|   false|
|[Market, at, Sans...|   false|
|[Temporary, Trans...|   false|
|[San, Francisco, ...|    true|
|[University, and,...|   false|
| [Spear, at, Folsom]|   false|
|[Temporary, Trans...|   false|
|[San, Francisco, ...|    true|
|[San, Francisco, ...|    true|
|[Mountain, View, ...|   false|
|[Steuart, at, Mar...|   false|
|[Yerba, Buena, Ce...|   false|
|[Harry, Bridges, ...|   false|
|[Embarcadero, at,...|   false|
| [Spear, at, Folsom]|   false|
|[San, Francisco, ...|    true|
|[Market

### Funkcje agregujące

In [0]:
from pyspark.sql.functions import avg, skewness, kurtosis

aggDf = df.select(avg("Duration"), skewness("Duration"), kurtosis("Duration"))

aggDf.show(50)

+------------------+------------------+------------------+
|     avg(Duration)|skewness(Duration)|kurtosis(Duration)|
+------------------+------------------+------------------+
|1046.0326611172604| 539.9225455582219|309473.43755760806|
+------------------+------------------+------------------+



## Zad 2

In [0]:
from pyspark.sql.functions import udf
from pyspark.sql.types import StringType

def fun2(string: str) -> str:
    return string.split()[0] if string else None

fun2_udf = udf(fun2, StringType())

fDf = df.select("Start Station", fun2_udf("Start Station").alias("First word"))

fDf.show(50)


+--------------------+-----------+
|       Start Station| First word|
+--------------------+-----------+
|Harry Bridges Pla...|      Harry|
|San Antonio Shopp...|        San|
|      Post at Kearny|       Post|
|  San Jose City Hall|        San|
|Embarcadero at Fo...|Embarcadero|
|Yerba Buena Cente...|      Yerba|
|Embarcadero at Fo...|Embarcadero|
|Embarcadero at Sa...|Embarcadero|
|     Beale at Market|      Beale|
|      Post at Kearny|       Post|
|Embarcadero at Sa...|Embarcadero|
|      Market at 10th|     Market|
|       Market at 4th|     Market|
|   Market at Sansome|     Market|
|Temporary Transba...|  Temporary|
|San Francisco Cal...|        San|
|University and Em...| University|
|     Spear at Folsom|      Spear|
|Temporary Transba...|  Temporary|
|San Francisco Cal...|        San|
|San Francisco Cal...|        San|
|Mountain View Cal...|   Mountain|
|   Steuart at Market|    Steuart|
|Yerba Buena Cente...|      Yerba|
|Harry Bridges Pla...|      Harry|
|Embarcadero at Sa..

In [0]:
from pyspark.sql.functions import pandas_udf
from pyspark.sql.types import IntegerType
import pandas as pd

# Definicja pandas_udf (mnożenie wartości przez 2)
@pandas_udf(returnType=IntegerType())
def fun(arr: pd.Series) -> pd.Series:
    return arr * 2

# Zastosowanie w DataFrame
ffDf = df.select("Duration", fun("Duration").alias("Double Duration"))

# Wyświetlenie
ffDf.show(50)


+--------+---------------+
|Duration|Double Duration|
+--------+---------------+
|     765|           1530|
|    1036|           2072|
|     307|            614|
|     409|            818|
|     789|           1578|
|     293|            586|
|     896|           1792|
|     255|            510|
|     126|            252|
|     932|           1864|
|     691|           1382|
|     633|           1266|
|     387|            774|
|     281|            562|
|     424|            848|
|     283|            566|
|     145|            290|
|     703|           1406|
|     605|           1210|
|     902|           1804|
|     481|            962|
|     335|            670|
|     704|           1408|
|     326|            652|
|     274|            548|
|     709|           1418|
|     486|            972|
|     273|            546|
|     497|            994|
|     395|            790|
|     509|           1018|
|     671|           1342|
|     307|            614|
|     359|            718|
|